In [288]:
import pandas as pd
import numpy as np
from StringIO import StringIO
from zipfile import ZipFile
from urllib import urlopen
from sklearn.linear_model import LogisticRegression
import statsmodels.formula.api as smf

import pandas as pd
import numpy as np
from StringIO import StringIO
from zipfile import ZipFile
from urllib import urlopen
import requests
import csv
import matplotlib.pylab as plt
import seaborn as sns
import pylab as pylab
from collections import OrderedDict
from sklearn.datasets import make_classification
from sklearn.datasets import make_blobs
from matplotlib.pylab import plt
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [289]:
def unzip_read_file(year):
    url = urlopen("http://www.nyc.gov/html/nypd/downloads/zip/analysis_and_planning/{}_sqf_csv.zip".format(year))
    zipfile = ZipFile(StringIO(url.read()))
    try:
        with zipfile.open("{}.csv".format(year)) as file:
            df = pd.read_csv(file)
            
    except: # 2015 csv name is different format than all the rest...go figure.
        with zipfile.open("{}_sqf_csv.csv".format(year)) as file:
            df = pd.read_csv(file)
    return df

In [290]:
df = pd.DataFrame()

In [291]:
# This will capture all data. Loading may take a while and will consume a fair amount of memory. 
# Recommended 16GB of memory if you want to run this cell...

# the rename is there to normalize the data from year to year. without,
# data of different years will not overlap properly

cols = []
#
for year in range(2003, 2004): #change this back to 2016 for all data
    year_df = unzip_read_file(year)
    year_df.rename(columns={'adrnum': 'addrnum', 'adrpct': 'addrpct', 'detailcm': 'detailCM', 
                            'dettyp_c': 'dettypCM', 'dettypcm': 'dettypCM', 'linecm': 'lineCM', 
                            'premtyp': 'premtype', 'prenam': 'prename', 'rescod': 'rescode',
                            'stinter': 'strintr', 'stname': 'strname'}, inplace=True)
    cols.append(list(year_df.columns))
    if len(list(year_df.columns)) != len(set(list(year_df.columns))):
        print sorted(list(year_df.columns))
    print year
    df = df.append(year_df)

2003


In [292]:
# early years missing detailCM, so excluded at this point...

# 'year', 'pct', 'datestop', 'timestop', 'age' , 'perobs'


sqf_primary_select = df.loc[:,['inout', 'explnstp', 'arstmade', 
      'sumissue', 'frisked', 'searched', 'contrabn',  
      'rf_vcrim', 'rf_othsw', 'rf_attir', 'cs_objcs', 'cs_descr', 
      'cs_casng', 'cs_lkout', 'rf_vcact', 'cs_cloth', 'cs_drgtr', 
      'cs_furtv', 'rf_rfcmp', 'rf_verbl', 'cs_vcrim', 'cs_bulge', 
      'cs_other',  'rf_knowl',  'sb_hdobj', 'sb_outln', 'sb_admis', 
      'sb_other', 'rf_furt', 'rf_bulg',  'sex', 'race']] # 'detailCM'

In [293]:
sqf_primary_select2 = df.loc[:,['year', 'inout', 'explnstp', 'arstmade', 
      'sumissue', 'frisked', 'searched', 'contrabn',  
      'rf_vcrim', 'rf_othsw', 'rf_attir', 'cs_objcs', 'cs_descr', 
      'cs_casng', 'cs_lkout', 'rf_vcact', 'cs_cloth', 'cs_drgtr', 
      'cs_furtv', 'rf_rfcmp', 'rf_verbl', 'cs_vcrim', 'cs_bulge', 
      'cs_other',  'rf_knowl',  'sb_hdobj', 'sb_outln', 'sb_admis', 
      'sb_other', 'rf_furt', 'rf_bulg',  'sex', 'race']]

In [294]:

print sqf_primary_select.columns

Index([u'inout', u'explnstp', u'arstmade', u'sumissue', u'frisked',
       u'searched', u'contrabn', u'rf_vcrim', u'rf_othsw', u'rf_attir',
       u'cs_objcs', u'cs_descr', u'cs_casng', u'cs_lkout', u'rf_vcact',
       u'cs_cloth', u'cs_drgtr', u'cs_furtv', u'rf_rfcmp', u'rf_verbl',
       u'cs_vcrim', u'cs_bulge', u'cs_other', u'rf_knowl', u'sb_hdobj',
       u'sb_outln', u'sb_admis', u'sb_other', u'rf_furt', u'rf_bulg', u'sex',
       u'race'],
      dtype='object')


In [295]:
def fix_nans(column):
    if column in [' ', '  ', '']:
        column = np.nan
    
    return column

In [296]:
for i in sqf_primary_select.columns:
    sqf_primary_select[i] = sqf_primary_select[i].apply(fix_nans)
    sqf_primary_select2[i] = sqf_primary_select2[i].apply(fix_nans)

In [297]:
# original data size
len(sqf_primary_select)

160851

In [298]:
# Dataframe after nan reduction
sqf_primary_select.dropna(inplace=True)
sqf_primary_select2.dropna(inplace=True)

len(sqf_primary_select)

159887

In [299]:

# Data cleaning to generate numeric values


d = {'Y': 1, 'N': 0}
sqf_primary_select = sqf_primary_select.replace(d)

In [319]:
#sqf_primary_select.head(5)

In [318]:
sqf_primary_select = pd.get_dummies(sqf_primary_select)
#sqf_primary_select.columns

In [302]:
#Linear Regression:

lm = smf.ols(formula = 'arstmade ~ explnstp + ' + ' + '.join(sqf_primary_select.columns[2:]), \
             data = sqf_primary_select).fit()


In [303]:
temp = pd.DataFrame(lm.params)
temp2 = pd.DataFrame(lm.pvalues)
printout = pd.concat([temp, temp2], axis = 1)
printout.columns = ['linear params', 'linear p-value']
printout['var'] = printout.index
print printout

           linear params  linear p-value        var
Intercept  -7.625254e+09    7.203782e-01  Intercept
explnstp   -2.267465e-02    4.928850e-04   explnstp
sumissue   -4.757060e-02    1.246119e-92   sumissue
frisked     3.739080e-02    1.320149e-91    frisked
searched    3.441688e-01    0.000000e+00   searched
contrabn    3.451732e-01    0.000000e+00   contrabn
rf_vcrim   -5.965762e-03    2.507098e-03   rf_vcrim
rf_othsw    9.315298e-03    1.521221e-07   rf_othsw
rf_attir   -2.391146e-02    3.295379e-18   rf_attir
cs_objcs    5.570705e-02    2.126523e-78   cs_objcs
cs_descr    1.119534e-04    9.379446e-01   cs_descr
cs_casng   -3.561120e-03    2.621641e-02   cs_casng
cs_lkout   -5.767903e-03    1.056677e-03   cs_lkout
rf_vcact   -6.855254e-03    1.077086e-02   rf_vcact
cs_cloth   -7.467663e-03    2.949983e-02   cs_cloth
cs_drgtr    7.528825e-03    3.349642e-05   cs_drgtr
cs_furtv   -2.215716e-03    1.280552e-01   cs_furtv
rf_rfcmp    4.955638e-03    1.510084e-02   rf_rfcmp
rf_verbl    

In [304]:
for i in range(32):
    if i != 2:
        lm = smf.ols(formula = 'arstmade ~ {}'.format(sqf_primary_select.columns[i]), data = sqf_primary_select).fit()
        print 'arstmade ~ ' + sqf_primary_select.columns[i] + ' + 1'
        temp = pd.DataFrame(lm.params)
        temp2 = pd.DataFrame(lm.pvalues)
        printout1 = pd.concat([temp, temp2], axis = 1)
        printout1.columns = ['params', 'p-value']
        print printout1

arstmade ~ explnstp + 1
             params       p-value
Intercept  0.132094  5.231900e-56
explnstp  -0.054707  7.451641e-11
arstmade ~ arstmade + 1
                 params  p-value
Intercept -2.247679e-14      0.0
arstmade   1.000000e+00      0.0
arstmade ~ frisked + 1
             params        p-value
Intercept  0.029193  2.526708e-233
frisked    0.105504   0.000000e+00
arstmade ~ searched + 1
             params  p-value
Intercept  0.030975      0.0
searched   0.528192      0.0
arstmade ~ contrabn + 1
             params  p-value
Intercept  0.062693      0.0
contrabn   0.631452      0.0
arstmade ~ rf_vcrim + 1
             params       p-value
Intercept  0.073696  0.000000e+00
rf_vcrim   0.038170  6.646245e-69
arstmade ~ rf_othsw + 1
             params  p-value
Intercept  0.055029      0.0
rf_othsw   0.140859      0.0
arstmade ~ rf_attir + 1
             params  p-value
Intercept  0.077682   0.0000
rf_attir   0.001297   0.6988
arstmade ~ cs_objcs + 1
             params        p-

In [305]:
#Correlation Matrix
sqf_primary_select.corr()

,explnstp,arstmade,sumissue,frisked,searched,contrabn,rf_vcrim,rf_othsw,rf_attir,cs_objcs,...,sex_Z,race_A,race_B,race_I,race_P,race_Q,race_U,race_W,race_X,race_Z
explnstp,1.000000,-0.016283,0.003139,-0.000436,-0.012300,-0.015260,0.002338,-0.005388,0.006116,-0.000835,...,-0.024347,-0.001397,0.006643,-0.000589,0.001435,0.002191,0.000284,0.004067,-0.008440,-0.016308
arstmade,-0.016283,1.000000,0.004944,0.196387,0.560366,0.359637,0.043854,0.193448,0.000968,0.070345,...,-0.025065,-0.003028,-0.002777,-0.000865,0.010137,0.009612,-0.001027,0.009302,-0.004892,-0.023673
sumissue,0.003139,0.004944,1.000000,0.058152,0.065633,0.073029,-0.025866,0.024578,0.009902,0.009238,...,-0.015651,0.005682,-0.008364,0.005437,0.010315,0.007763,-0.000844,0.009207,-0.004830,-0.017109
frisked,-0.000436,0.196387,0.058152,1.000000,0.290420,0.108412,0.352733,0.360941,0.217232,-0.021035,...,-0.015849,-0.042731,0.054856,-0.012566,0.036464,0.003792,0.003831,-0.079368,-0.016809,-0.010267
searched,-0.012300,0.560366,0.065633,0.290420,1.000000,0.287861,0.071205,0.232203,0.042138,0.048533,...,-0.030528,-0.005673,0.007789,0.003789,0.009670,0.012411,-0.001102,-0.007113,-0.008458,-0.025887
contrabn,-0.015260,0.359637,0.073029,0.108412,0.287861,1.000000,0.003982,0.091899,-0.003625,0.058269,...,-0.015686,-0.001973,-0.002976,0.001810,0.007861,0.008080,-0.000553,0.005226,-0.004961,-0.015812
rf_vcrim,0.002338,0.043854,-0.025866,0.352733,0.071205,0.003982,1.000000,-0.006743,0.014462,-0.025759,...,-0.020727,-0.008571,0.030818,-0.005786,0.014402,0.004793,0.004532,-0.036013,-0.010283,-0.023027
rf_othsw,-0.005388,0.193448,0.024578,0.360941,0.232203,0.091899,-0.006743,1.000000,-0.022614,0.013907,...,-0.002427,0.006731,-0.000726,-0.003059,0.004650,0.005269,-0.001551,-0.012822,-0.002124,0.001713
rf_attir,0.006116,0.000968,0.009902,0.217232,0.042138,-0.003625,0.014462,-0.022614,1.000000,-0.000320,...,0.001057,-0.015583,0.004436,-0.001286,0.022078,-0.000063,-0.000737,-0.022045,-0.006809,0.011191
cs_objcs,-0.000835,0.070345,0.009238,-0.021035,0.048533,0.058269,-0.025759,0.013907,-0.000320,1.000000,...,0.001821,0.014137,-0.033916,0.004689,0.002297,0.015729,-0.000647,0.020135,0.001780,0.003154


In [306]:
x1 = sqf_primary_select.iloc[:, 0:2]
x2 = sqf_primary_select.iloc[:, 3:]
X = pd.concat([x1, x2], axis = 1)

In [307]:
##Logistic Regression

#learn logistic regression over the training sample
logit = LogisticRegression(C=1e6)
logit.fit(np.c_[X], sqf_primary_select['arstmade'])
# C here controls the regularization strength--which will be discussed in Bayesian session
# For now just keep in mind that setting a high value will make the regularization ineffective


LogisticRegression(C=1000000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [308]:
print "coefficients of logistic regression: \n", 
logistic = []
print "Intercept ", + logit.intercept_[0]
logistic.append(["Intercept", logit.intercept_[0]])
for i in range(X.shape[1]):
    logistic.append([X.columns[i], logit.coef_[0][i]])
    print X.columns[i], + logit.coef_[0][i]


coefficients of logistic regression: 
Intercept  -4.94124460575
explnstp -3.10953067192
arstmade 24.2734789195
frisked 0.315598341627
searched 1.07003161872
contrabn 1.12039369027
rf_vcrim 0.063566607292
rf_othsw 0.149287752416
rf_attir -0.0636859917354
cs_objcs 0.252861504304
cs_descr -0.149600994675
cs_casng -0.331062496152
cs_lkout -0.26415973113
rf_vcact 0.0203175066216
cs_cloth -0.138925955435
cs_drgtr -0.186631476803
cs_furtv -0.248956238247
rf_rfcmp 0.186709515259
rf_verbl 0.0431417795204
cs_vcrim -0.0850012570109
cs_bulge -0.367505874704
cs_other -0.179099613416
rf_knowl 0.173309928067
sb_hdobj -0.0116689741943
sb_outln 0.0910338299914
sb_admis 0.189827766762
sb_other 0.787989253445
rf_furt 0.0399908539445
rf_bulg -0.214514759116
inout_I -2.3475893509
inout_O -2.59365525485
sex_F -1.46137726826
sex_M -1.85023269652
sex_Z -1.62963464097
race_A -0.487455304463
race_B -0.842382172461
race_I -0.101674182675
race_P -0.593247667793
race_Q -0.805481459377
race_U -0.000541143237385
rac

In [317]:
logistic = pd.DataFrame(logistic)
logistic.columns = ['var', 'logistic params']
#print logistic

In [315]:
loglinreg = logistic.merge(printout, on ="var")

In [316]:
loglinreg

,var,logistic params,linear params,linear p-value
0,Intercept,-4.941245,-7.625254e+09,7.203782e-01
1,explnstp,-3.109531,-2.267465e-02,4.928850e-04
2,frisked,0.315598,3.739080e-02,1.320149e-91
3,searched,1.070032,3.441688e-01,0.000000e+00
4,contrabn,1.120394,3.451732e-01,0.000000e+00
5,rf_vcrim,0.063567,-5.965762e-03,2.507098e-03
6,rf_othsw,0.149288,9.315298e-03,1.521221e-07
7,rf_attir,-0.063686,-2.391146e-02,3.295379e-18
8,cs_objcs,0.252862,5.570705e-02,2.126523e-78
9,cs_descr,-0.149601,1.119534e-04,9.379446e-01
